In [ ]:
import os, sys
from os.path import abspath, dirname
import zarr
import z5py
import numpy as np
import pandas as pd
from glob import glob 
from skimage.measure import regionprops
from skimage.io import imread, imsave
from scipy import stats
from scipy.stats import skewnorm, lognorm
from scipy.optimize import minimize
import itertools
import collections

sys.path.append('../../')
from easi_fish import n5_metadata_utils as n5mu
from easi_fish import roi_prop, spot, intensity
import warnings
warnings.filterwarnings('ignore')

import importlib
importlib.reload(spot)
importlib.reload(roi_prop)
importlib.reload(intensity)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

sns.set_style('white', rc={'axes.grid':True})
sns.set_context('talk')

In [ ]:
## input
ddir = '/u/home/f/f7xiesnm/project-zipursky/easifish/'

# spot dir for every gene
# all Sorcs3 channel c0
fx_spots1 = {
    'lt172 (no flatfield)':      ddir+'lt172/outputs/r1/spots/spots_c0.txt',
    'sparse06':        ddir+'/sparse06/outputs/r1_flatfield/v8/spots/spots_c0.txt',
    'lt186':           ddir+'/lt186/outputs/r2v3/spots/spots_c0.txt',
    'lt185':           ddir+'/lt185/outputs/r2v3/spots/spots_c0.txt',
    'cdf03_c1-2_bino': ddir+'/cdf03_c1-2_bino/outputs/r3/spots/spots_c0.txt',
    'cdf04_c1-2_bino': ddir+'/cdf04_c1-2_bino/outputs/r3/spots/spots_c0.txt',
}

fx_spots2 = {
    'lt186 (r1c0)': ddir+'/lt186/outputs/r1v3/spots/spots_c0.txt',
    'lt186 (r1c2)': ddir+'/lt186/outputs/r1v3/spots/spots_c2.txt',
    
    'lt186 (r2c0)': ddir+'/lt186/outputs/r2v3/spots/spots_c0.txt',
    'lt186 (r2c1)': ddir+'/lt186/outputs/r2v3/spots/spots_c1.txt',
    'lt186 (r2c2)': ddir+'/lt186/outputs/r2v3/spots/spots_c2.txt',
    'lt186 (r2c4)': ddir+'/lt186/outputs/r2v3/spots/spots_c4.txt',
    
    'lt186 (r3c0)': ddir+'/lt186/outputs/r3v3/spots/spots_c0.txt',
    'lt186 (r3c1)': ddir+'/lt186/outputs/r3v3/spots/spots_c1.txt',
    'lt186 (r3c2)': ddir+'/lt186/outputs/r3v3/spots/spots_c2.txt',
    'lt186 (r3c4)': ddir+'/lt186/outputs/r3v3/spots/spots_c4.txt',
    
    'lt186 (r4c0)': ddir+'/lt186/outputs/r4v3/spots/spots_c0.txt',
    'lt186 (r4c1)': ddir+'/lt186/outputs/r4v3/spots/spots_c1.txt',
    'lt186 (r4c2)': ddir+'/lt186/outputs/r4v3/spots/spots_c2.txt',
    'lt186 (r4c4)': ddir+'/lt186/outputs/r4v3/spots/spots_c4.txt',
    
    'lt186 (r6c0)': ddir+'/lt186/outputs/r6v3/spots/spots_c0.txt',
    'lt186 (r6c1)': ddir+'/lt186/outputs/r6v3/spots/spots_c1.txt',
    'lt186 (r6c2)': ddir+'/lt186/outputs/r6v3/spots/spots_c2.txt',
    'lt186 (r6c4)': ddir+'/lt186/outputs/r6v3/spots/spots_c4.txt',
    
    'lt186 (r7c0)': ddir+'/lt186/outputs/r7v3/spots/spots_c0.txt',
    'lt186 (r7c1)': ddir+'/lt186/outputs/r7v3/spots/spots_c1.txt',
    'lt186 (r7c2)': ddir+'/lt186/outputs/r7v3/spots/spots_c2.txt',
    'lt186 (r7c4)': ddir+'/lt186/outputs/r7v3/spots/spots_c4.txt',
}
colors = {name: f'C{int(name.split("(")[1][1])-1}' for name, file in fx_spots2.items()}
print(colors)

fx_spots3 = {
    'cdf03 (r1c0)': ddir+'/cdf03_c1-2_bino/outputs/r1/spots/spots_c0.txt',
    'cdf03 (r1c2)': ddir+'/cdf03_c1-2_bino/outputs/r1/spots/spots_c2.txt',
    
    'cdf03 (r3c0)': ddir+'/cdf03_c1-2_bino/outputs/r3/spots/spots_c0.txt',
    'cdf03 (r3c1)': ddir+'/cdf03_c1-2_bino/outputs/r3/spots/spots_c1.txt',
    'cdf03 (r3c2)': ddir+'/cdf03_c1-2_bino/outputs/r3/spots/spots_c2.txt',
    'cdf03 (r3c4)': ddir+'/cdf03_c1-2_bino/outputs/r3/spots/spots_c4.txt',
}

fx_spots4 = {
    'sparse06 (r1c0)': ddir+'/sparse06/outputs/r1_flatfield/v8/spots/spots_c0.txt',
    'sparse06 (r1c2)': ddir+'/sparse06/outputs/r1_flatfield/v8/spots/spots_c2.txt',
    'sparse06 (r1c4)': ddir+'/sparse06/outputs/r1_flatfield/v8/spots/spots_c4.txt',
}

fx_spots5 = {
    'lt172 (r1c0)': ddir+'/lt172/outputs/r1/spots/spots_c0.txt',
    'lt172 (r1c1)': ddir+'/lt172/outputs/r1/spots/spots_c1.txt',
    'lt172 (r1c2)': ddir+'/lt172/outputs/r1/spots/spots_c2.txt',
    'lt172 (r1c4)': ddir+'/lt172/outputs/r1/spots/spots_c4.txt',
    
    'lt172 (r2c0)': ddir+'/lt172/outputs/r2/spots/spots_c0.txt',
    'lt172 (r2c1)': ddir+'/lt172/outputs/r2/spots/spots_c1.txt',
    'lt172 (r2c2)': ddir+'/lt172/outputs/r2/spots/spots_c2.txt',
    'lt172 (r2c4)': ddir+'/lt172/outputs/r2/spots/spots_c4.txt',
    
    'lt172 (r3c0)': ddir+'/lt172/outputs/r3/spots/spots_c0.txt',
    'lt172 (r3c1)': ddir+'/lt172/outputs/r3/spots/spots_c1.txt',
    'lt172 (r3c2)': ddir+'/lt172/outputs/r3/spots/spots_c2.txt',
    'lt172 (r3c4)': ddir+'/lt172/outputs/r3/spots/spots_c4.txt',
    
    'lt172 (r4c0)': ddir+'/lt172/outputs/r4/spots/spots_c0.txt',
    'lt172 (r4c1)': ddir+'/lt172/outputs/r4/spots/spots_c1.txt',
    'lt172 (r4c2)': ddir+'/lt172/outputs/r4/spots/spots_c2.txt',
    'lt172 (r4c4)': ddir+'/lt172/outputs/r4/spots/spots_c4.txt',
    
    'lt172 (r5c0)': ddir+'/lt172/outputs/r5/spots/spots_c0.txt',
    'lt172 (r5c1)': ddir+'/lt172/outputs/r5/spots/spots_c1.txt',
    'lt172 (r5c2)': ddir+'/lt172/outputs/r5/spots/spots_c2.txt',
    'lt172 (r5c4)': ddir+'/lt172/outputs/r5/spots/spots_c4.txt',
}

for name, f in fx_spots1.items():
    assert os.path.isfile(f)
    
for name, f in fx_spots2.items():
    assert os.path.isfile(f)
    
for name, f in fx_spots3.items():
    assert os.path.isfile(f)
    
for name, f in fx_spots4.items():
    assert os.path.isfile(f)
    
for name, f in fx_spots5.items():
    assert os.path.isfile(f)

In [ ]:
def pipe(fx_spots):
    """
    """
    dfs = {} 
    for i, (name, file) in enumerate(fx_spots.items()):
        df = pd.read_csv(file, header=None)
        if name.startswith('lt172'):
            df[3] = df[3]-90
            df = df[df[3] > 5]
        else:
            df = df[df[3] > 5]
        if i == 0:
            num0 = len(df)

        print(f'{len(df)/num0:.1f}', name)
        dfs[name] = df
    return dfs

In [ ]:
def plot1(dfs, xlim=None, cumulative=False, colors=None):
    fig, ax = plt.subplots()
    for name, df in dfs.items():
        med = np.median(df[3])
        if colors is not None:
            color = colors[name]
        else:
            color = None
        if cumulative: 
            sns.histplot(df[3], ax=ax, bins=bins, label=f'med={med:.1f} ({name})', stat='percent', element='step', 
                         color=color,
                         alpha=1, fill=False, cumulative=True)
        else:
            sns.histplot(df[3], ax=ax, bins=bins, label=f'med={med:.1f} ({name})', stat='percent', element='step', 
                         color=color,
                         alpha=0.1, fill=True)
            
    ax.set_xlabel('spot intensity') #0
    ax.legend(bbox_to_anchor=(1,1), loc='upper left')
    ax.set_title('spot intensity <5 are excluded')
    
    if xlim is not None:
        ax.set_xlim(xlim)
    plt.show()

In [ ]:
dfs1 = pipe(fx_spots1)
dfs2 = pipe(fx_spots2)
dfs3 = pipe(fx_spots3)
dfs4 = pipe(fx_spots4)
dfs5 = pipe(fx_spots5)

In [ ]:
bins = np.linspace(0, 200, 200*2)

In [ ]:
plot1(dfs1)
plot1(dfs1, xlim=[0,50])
plot1(dfs1, xlim=[0,50], cumulative=True)

In [ ]:
plot1(dfs2, colors=colors)
plot1(dfs3)
plot1(dfs4)
plot1(dfs5)

# plot1(dfs2, xlim=[0,50], colors=colors)
# plot1(dfs2, xlim=[0,50], cumulative=True, colors=colors)

# plot1(dfs3, xlim=[0,50])
# plot1(dfs3, xlim=[0,50], cumulative=True)

In [ ]:
plot1(dfs1, xlim=[0,50], cumulative=True)
plot1(dfs2, xlim=[0,50], cumulative=True)
plot1(dfs3, xlim=[0,50], cumulative=True)
plot1(dfs4, xlim=[0,50], cumulative=True)
plot1(dfs5, xlim=[0,50], cumulative=True)